# XGB with data cleaning and feature engineering.

In [ ]:
sub.to_csv('xgb_starter.csv.gz', index=False, float_format='%.4g', compression='gzip')

## Intro:

In this notebook, I will try to a) import and clean data and b) run an XGB decision tree regression model to make Kaggle predictions.

Shoutout to these helpful scripts:

- https://www.kaggle.com/infinitewing/xgboost-without-outliers-lb-0-06463

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import re

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

In [3]:
np.random.seed(1234)

In [4]:
import timeit

In [50]:
# Read data from saved pickle files.
start_time = timeit.default_timer()
properties = pd.read_pickle('Data/properties')
train = pd.read_pickle('Data/train')
elapsed = timeit.default_timer() - start_time

In [51]:
train_df = train.merge(properties, how='left', on='parcelid')

In [6]:
print('Pickle import only takes {0:.3g} seconds...a big improvement over reading CSV files.'.format(elapsed))

Pickle import only takes 4.33 seconds...a big improvement over reading CSV files.


In [7]:
#properties.info()

** Most of the variables are just integers...so maybe we should recode to save space? **

The properties file contains data on {{properties.shape[0]}} housing properties.

## XGB model fit

In [ ]:


# xgboost params
xgb_params = {
    'eta': 0.06,
    'max_depth': 5,
    'subsample': 0.75,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

In [ ]:


# cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   nfold=5,
                   num_boost_round=200,
                   early_stopping_rounds=50,
                   verbose_eval=10, 
                   show_stdv=False
                  )
num_boost_rounds = len(cv_result)
print(num_boost_rounds)

In [ ]:
# train model
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)
pred = model.predict(dtest)
y_pred=[]

In [ ]:
for i,predict in enumerate(pred):
    y_pred.append(str(round(predict,4)))

In [ ]:
y_pred=np.array(y_pred)

In [ ]:


output = pd.DataFrame({'ParcelId': properties['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})

In [ ]:
output.head()

In [ ]:

# set col 'ParceID' to first col
cols = output.columns.tolist()

In [ ]:
cols

In [ ]:
cols = cols[-1:] + cols[:-1]
cols

In [ ]:
output = output[cols]

In [ ]:
output.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%_m_%d_%H:%M:%S')), index=False)

In [ ]:
sub.to_csv('xgb_starter.csv.gz', index=False, float_format='%.4g', compression='gzip')